In [ ]:
from scripts.flink_sql_gateway_wrapper import connect, sql, close
from time import sleep

In [ ]:
connect("localhost", 8081)

In [ ]:
sql("""CREATE CATALOG IF NOT EXISTS paimon_catalog WITH (
  'type' = 'paimon',
  'warehouse' = 's3://paimon-data/paimon-warehouse',
  's3.endpoint' = 'http://seaweedfs-s3:8333',
  's3.access-key' = 'paimonAdmin123',
  's3.secret-key' = 'paimonSecretKey456789abcdef',
  's3.path-style-access' = 'true'
);""")

In [ ]:
sql("USE CATALOG paimon_catalog")
sql("""CREATE DATABASE IF NOT EXISTS testdb;""")
sql("""USE testdb;""")

In [ ]:
sql("""
CREATE TEMPORARY TABLE IF NOT EXISTS mysql_users_batch (
    id INT,
    name STRING,
    email STRING,
    age INT,
    created_at TIMESTAMP(3),
    updated_at TIMESTAMP(3),
    PRIMARY KEY (id) NOT ENFORCED
) WITH (
    'connector' = 'jdbc',
    'url' = 'jdbc:mysql://mysql:3306/testdb',
    'table-name' = 'users',
    'username' = 'flink',
    'password' = 'flink123'
)
""")
sql("""SELECT * FROM mysql_users_batch;""")

In [ ]:
sql("""CREATE TEMPORARY TABLE IF NOT EXISTS mysql_users (
  id INT,
  name STRING,
  email STRING,
  age INT,
  created_at TIMESTAMP(3),
  updated_at TIMESTAMP(3),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'connector' = 'mysql-cdc',
  'hostname' = 'mysql',
  'port' = '3306',
  'username' = 'flink',
  'password' = 'flink123',
  'database-name' = 'testdb',
  'table-name' = 'users',
  'scan.startup.mode' = 'initial'
);""")

In [ ]:
sql("""
CREATE TEMPORARY TABLE IF NOT EXISTS mysql_orders_batch (
    id INT,
    user_id INT,
    product_name STRING,
    quantity INT,
    price DECIMAL(10, 2),
    order_date TIMESTAMP(3),
    PRIMARY KEY (id) NOT ENFORCED
) WITH (
    'connector' = 'jdbc',
    'url' = 'jdbc:mysql://mysql:3306/testdb',
    'table-name' = 'orders',
    'username' = 'flink',
    'password' = 'flink123'
)
""")
sql("""SELECT * FROM mysql_users_batch;""")

In [ ]:
sql("""CREATE TEMPORARY TABLE IF NOT EXISTS mysql_orders (
  id INT,
  user_id INT,
  product_name STRING,
  quantity INT,
  price DECIMAL(10, 2),
  order_date TIMESTAMP(3),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'connector' = 'mysql-cdc',
  'hostname' = 'mysql',
  'port' = '3306',
  'username' = 'flink',
  'password' = 'flink123',
  'database-name' = 'testdb',
  'table-name' = 'orders',
  'scan.startup.mode' = 'initial'
);""")

In [ ]:
sql("""CREATE TABLE IF NOT EXISTS users (
  id INT,
  name STRING,
  email STRING,
  age INT,
  created_at TIMESTAMP(4),
  updated_at TIMESTAMP(4),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'metadata.iceberg.storage' = 'table-location',
  'data-file.path-directory' = 'data'
);""")
sql("""INSERT INTO users SELECT * FROM mysql_users;""")

In [ ]:
sql("SELECT * FROM users")

In [ ]:
sql("""CREATE TABLE IF NOT EXISTS orders (
  id INT,
  user_id INT,
  product_name STRING,
  quantity INT,
  price DECIMAL(10, 2),
  order_date TIMESTAMP(4),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'metadata.iceberg.storage' = 'table-location',
  'data-file.path-directory' = 'data'
);""")
sql("""INSERT INTO orders SELECT * FROM mysql_orders;""")

In [ ]:
sql("SELECT * FROM orders")

In [ ]:
close()